Which Legendre Polynomials Should We Use
------

I am having trouble constructing the constraint matrix on the diagonal face. The top and bottom are easy, that is done. The question is: which tensor product Legendre polynomials do I use on the diagonal face?

In [1]:
from sympy import *
import numpy as np
x,y,z = symbols('x y z')

First I will define the 2nd and 3rd order tensor product Legendre polynomials.

In [2]:
L = []
L.append(legendre(2,x)*legendre(0,z)) # n = 2
L.append(legendre(1,x)*legendre(1,z))
L.append(legendre(0,x)*legendre(2,z))
L.append(legendre(3,x)*legendre(0,z)) # n = 3
L.append(legendre(2,x)*legendre(1,z))
L.append(legendre(1,x)*legendre(2,z))
L.append(legendre(0,x)*legendre(3,z))

Then I will define the full hexahedral basis as in (Wheeler, Xue, & Yotov, 2012)

In [3]:
def curl(v):
    c = []
    c.append(  diff(v[2],y)-diff(v[1],z) )
    c.append(-(diff(v[2],x)-diff(v[0],z)))
    c.append(  diff(v[1],x)-diff(v[0],y) )
    return np.asarray(c)

P = np.asarray([[1,0,0],
                [x,0,0],
                [y,0,0],
                [z,0,0],
                [0,1,0],
                [0,x,0],
                [0,y,0],
                [0,z,0],
                [0,0,1],
                [0,0,x],
                [0,0,y],
                [0,0,z]])
P = np.vstack([P,curl([0       ,0       ,x*y*z   ])])
P = np.vstack([P,curl([0       ,0       ,x*y**2  ])])
P = np.vstack([P,curl([0       ,0       ,x**2*z  ])])
P = np.vstack([P,curl([0       ,0       ,x**2*y*z])])
P = np.vstack([P,curl([x*y*z   ,0       ,0       ])])
P = np.vstack([P,curl([y*z**2  ,0       ,0       ])])
P = np.vstack([P,curl([x*y**2  ,0       ,0       ])])
P = np.vstack([P,curl([x*y**2*z,0       ,0       ])])
P = np.vstack([P,curl([0       ,x*y*z   ,0       ])])
P = np.vstack([P,curl([0       ,x**2*z  ,0       ])])
P = np.vstack([P,curl([0       ,y*z**2  ,0       ])])
P = np.vstack([P,curl([0       ,x*y*z**2,0       ])])

In order to see what space we span on the diagonal, I dot the space with the outward unit normal of the diagonal face and then set $y=-x$ (is there something wrong with this approach? Or do I need to rotate the coordinate system?). Then I will print each basis in order to see which terms we need to eliminate with the constraint matrix of (Kirby 2004).

In [4]:
diag = np.dot(P,[1,1,0])
for i in range(diag.size):
    if type(diag[i]) == type(0): continue
    diag[i] = diag[i].subs({y:-x})
    print("%2d %18s" % (i,diag[i]))

 1                  x
 2                 -x
 3                  z
 5                  x
 6                 -x
 7                  z
 9                  0
10                  0
11                  0
12              2*x*z
13            -3*x**2
14             -2*x*z
15           3*x**2*z
16              -x**2
17             -2*x*z
18                  0
19               x**3
20               x**2
21              -x**2
22              2*x*z
23           2*x**2*z


From this list, I see $x^2$ terms in bases 13, 16, 20, and 21, $x^2z$ terms in 15 and 23, and $x^3$ terms in basis 19. Like Jed said in his mail, I don't see the terms $z^2$ or $x z^2$. So as I understand it, I need to find Legendre polynomials which are not orthogonal to these terms, but are orthogonal to $1,x,z,xz$. So I will just brute force this by checking the integral is 0.

In [5]:
ind = []
for i in [13,15,16,19,20,21,23]:
    for j in range(len(L)):
        I = integrate(integrate(diag[i]*L[j],(x,-1,1)),(z,-1,1))
        if I == 0: continue  # orthogonal
        if j not in ind: ind.append(j)
print("The following set of Legendre polynomials are needed:")
for i in ind:
    print(L[i])


The following set of Legendre polynomials are needed:
3*x**2/2 - 1/2
z*(3*x**2/2 - 1/2)
5*x**3/2 - 3*x/2


So from this I see that 3 Legendre polynomials are sufficient to remove the unwanted terms. This, plus the two on the top faces, leaves me with 5 constraint equations. This is still one fewer than I expected. Have I made a mistake? From where might the last equation come? 